In [1]:
import numpy as np
import torch
from collections import deque
from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
from multiagent import MADDPG

## Load environment

In [6]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe", no_graphics=False)

# get env info
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name] 
num_agents = len(env_info.agents)

# size of each action
action_size = brain.vector_action_space_size

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


## Load trained agents

In [3]:
multiagent = MADDPG(state_size, action_size, num_agents)
multiagent.agents[0].load_state_dict(torch.load("savedata/agent1_episode2119.save"))
multiagent.agents[1].load_state_dict(torch.load("savedata/agent2_episode2119.save"))

## Observe agents

In [7]:
n_episodes = 5
max_t = 5000
for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations
    multiagent.reset()
    score = np.zeros(num_agents)
    steps = 0
    for t in range(max_t):
        action = multiagent.act(state)
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        multiagent.step(state, action, rewards, next_state, dones)
        state = next_state
        score += rewards
        steps+=1
        if np.any(dones):
            break 

In [8]:
env.close()